In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [14]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [16]:
x_true = tf.placeholder(tf.float32, [None, 28,28,1])
y_true = tf.placeholder(tf.float32, [None,10])

In [17]:
weight_1 = tf.Variable(tf.truncated_normal([3, 3,1,32], stddev=0.01))
hidden_1 = tf.nn.conv2d(x_true,weight_1, strides=[1,1,1,1],padding='SAME')
hidden_1 = tf.nn.relu(hidden_1)
hidden_1 = tf.nn.max_pool(hidden_1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
hidden_1 = tf.nn.dropout(hidden_1, keep_prob=0.8)

In [18]:
weight_2 = tf.Variable(tf.truncated_normal([3, 3,32,64], stddev=0.01))
hidden_2 = tf.nn.conv2d(hidden_1,weight_2, strides=[1,1,1,1],padding='SAME')
hidden_2 = tf.nn.relu(hidden_2)
hidden_2 = tf.nn.max_pool(hidden_2, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [19]:
hidden_2 = tf.reshape(hidden_2, [-1, 7*7*64])
hidden_2 = tf.nn.dropout(hidden_2, keep_prob=0.8)

In [20]:
weight_3 = tf.Variable(tf.truncated_normal([7*7*64, 256],stddev=0.01))
fc_1 = tf.matmul(hidden_2, weight_3)
fc_1 = tf.nn.relu(fc_1)
fc_1 = tf.nn.dropout(fc_1,0.5)

In [21]:
weight_4 = tf.Variable(tf.truncated_normal([256, 10],stddev=0.01))
y_pred = tf.matmul(fc_1, weight_4)

In [22]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred,labels=y_true)
cost = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimizer = optimizer.minimize(cost)

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [24]:
batch_size = 100
num_batch = int(mnist.train.num_examples /batch_size)
num_batch

550

In [25]:
for epoch in range(10):
    total_cost =0
    for i in range(num_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1,28,28,1)
        _, cost_value = sess.run([optimizer,cost],{x_true:batch_xs,y_true:batch_ys})
        total_cost += cost_value
    print("Epoch : {0}, Cost :{1}".format(epoch +1, total_cost/ num_batch))
print("최적화를 완료했습니다.")

Epoch : 1, Cost :0.255485615872524
Epoch : 2, Cost :0.13771518851715056
Epoch : 3, Cost :0.12304228668355129
Epoch : 4, Cost :0.11783593319525773
Epoch : 5, Cost :0.12024220111695202
Epoch : 6, Cost :0.11249183411286635
Epoch : 7, Cost :0.10807067625478588
Epoch : 8, Cost :0.11172409425997598
Epoch : 9, Cost :0.10639627350832928
Epoch : 10, Cost :0.10864399339580401
최적화를 완료했습니다.


In [28]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("정확도:{0}".format(sess.run(accuracy, {x_true:mnist.test.images.reshape(-1,28,28,1),y_true:mnist.test.labels})))

정확도:0.9688000082969666
